In [6]:
%pip install mesa
%pip install ggplot
%pip install networkx


In [2]:
import seaborn as sns

In [3]:
from mesa import Agent, Model

class MoneyAgent(Agent):
    """An agent with fixed initial wealth."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 1

class MoneyModel(Model):
    """A model with some number of agents."""
    def __init__(self, N):
        self.num_agents = N
        # Create agents
        for i in range(self.num_agents):
            a = MoneyAgent(i, self)

In [16]:
from mesa import Agent, Model
from mesa.time import RandomActivation,SimultaneousActivation,StagedActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

class FishingBoat(Agent):
    def __init__(self, name, model):
        super().__init__(name, model)
        self.name = name
        self.ndec=0

    def step(self):
        if self.ndec<1:
          print("{} activated".format(self.name))
        else:
          print(f"{self.name} with {self.pos} location")
        self.ndec+=1
        other_agent = self.random.choice(self.model.schedule.agents)

    def move(self):
      possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True, # Von Neumann excludes diagonal neighbourhood on the grid
            include_center=False)
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)        

class MyModel(Model):
    def __init__(self, n_agents):
        super().__init__()
        self.schedule = SimultaneousActivation(self)
        self.grid = MultiGrid(10, 10, torus=True)
        for i in range(n_agents):
            a = FishingBoat(i, self)
            self.schedule.add(a)
            coords = (self.random.randrange(0, 10), self.random.randrange(0, 10))
            self.grid.place_agent(a, coords)
        self.dc = DataCollector(model_reporters={"agent_count":
                                    lambda m: m.schedule.get_agent_count()},
                                agent_reporters={"name": lambda a: a.name})

    def step(self):
        self.schedule.step()
        self.dc.collect(self)

In [17]:
model = MyModel(5)
for t in range(10):
    model.step()
model_df = model.dc.get_model_vars_dataframe()
agent_df = model.dc.get_agent_vars_dataframe()

0 activated
1 activated
2 activated
3 activated
4 activated


In [ ]:
from mesa.batchrunner import BatchRunner
parameters = {"n_agents": range(1, 20)}
batch_run = BatchRunner(MyModel, parameters, max_steps=10,
                        model_reporters={"n_agents": lambda m: m.schedule.get_agent_count()})
batch_run.run_all()
batch_df = batch_run.get_model_vars_dataframe()

In [23]:
batch_df

,n_agents,Run
0,1,0
1,2,1
2,3,2
3,4,3
4,5,4
5,6,5
6,7,6
7,8,7
8,9,8
9,10,9


In [18]:
model_df

,agent_count
0,5
1,5
2,5
3,5
4,5
5,5
6,5
7,5
8,5
9,5


In [19]:
agent_df

name
Step AgentID      
1    0           0
     1           1
     2           2
     3           3
     4           4
2    0           0
     1           1
     2           2
     3           3
     4           4
3    0           0
     1           1
     2           2
     3           3
     4           4
4    0           0
     1           1
     2           2
     3           3
     4           4
5    0           0
     1           1
     2           2
     3           3
     4           4
6    0           0
     1           1
     2           2
     3           3
     4           4
7    0           0
     1           1
     2           2
     3           3
     4           4
8    0           0
     1           1
     2           2
     3           3
     4           4
9    0           0
     1           1
     2           2
     3           3
     4           4
10   0           0
     1           1
     2           2
     3           3
     4           4